In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.colors import colorConverter
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d import Axes3D
import os
import sys
import cbm_pack.cbm_pack as cbm_pack

from matplotlib import rcParams


In [5]:
extensions = {
    "mf_raster" : ".mfr",
    "gr_raster" : ".bin",
    "go_raster" : ".gor",
    "pc_raster" : ".pcr",
    "bc_raster" : ".bcr",
    "sc_raster" : ".scr",
    "nc_raster" : ".ncr",
    "io_raster" : ".ior",
    "mf_psth" : ".mfp", 
    "gr_psth" : ".grp",
    "go_psth" : ".gop",
    "pc_psth" : ".pcp",
    "bc_psth" : ".bcp",
    "sc_psth" : ".scp",
    "nc_psth" : ".ncp",
    "io_psth" : ".iop",
    "pfpc_weights" : ".pfpcw",
    "mfnc_weights" : ".mfncw"
}

cell_nums = {
    "mf" : 4096,
    "gr" : 2 ** 20,
    "go" : 4096,
    "pc" : 32,
    "bc" : 128,
    "sc" : 512,
    "nc" : 8,
    "io" : 4
}

MAX_CR_AMP = 6
PRE_CS_COLLECT_TS = 400
POST_CS_COLLECT_TS = 400

NUM_PROBE_TRIALS = 50 

# INPUT_DIR = "./data/inputs/"




# INPUT_NC_FILE = INPUT_DIR + INPUT_BASE + "/" + INPUT_BASE + extensions["nc_raster"]

In [6]:
def calc_cr_amp_per_isi(isis: np.ndarray, input_dir, basename=''):

    for isi in isis:
        isi_crs = []
  
        reset_base = f"{basename}_{isi}"
        reset_pc_file = f"{input_dir}{reset_base}/{reset_base}{extensions['pc_raster']}"

        if isi < 1000:
            NUM_TRIALS = 500
        else:
            NUM_TRIALS = 1000

        num_collect_ts = PRE_CS_COLLECT_TS + isi + POST_CS_COLLECT_TS
        pc_rasters = cbm_pack.np_arr_from_file(reset_pc_file, np.uint8)
        pc_rasters = cbm_pack.reshape_raster( \
                pc_rasters, \
                cell_nums["pc"], \
                NUM_TRIALS, \
                num_collect_ts)

        pc_reset_crs = cbm_pack.pcs_to_crs(pc_rasters, PRE_CS_COLLECT_TS, POST_CS_COLLECT_TS, isi)

        crs_per_ts = np.max(pc_reset_crs, axis=1)

            

        isi_crs.append(crs_per_ts)
        print(f"ISI {isi}: computed cr for final cr: {crs_per_ts[-1]}")
            
    return isi_crs
    
    # final setup stuff

In [7]:
ISIS = [1000, 1250, 1750]

FULL_INPUT_DIR= f"/home/satvik/lobule_v/CbmSim/data/outputs/forgetting_bugaloo/acq_no_probe_ISI/"
FULL_BASENAME=f"acq_no_probe_ISI"

FULL_CRS = calc_cr_amp_per_isi(ISIS, input_dir=FULL_INPUT_DIR, basename=FULL_BASENAME)